In [1]:
#  -------------------------------------------
"""
입출력 양식을
준수해 주십시오

###  INPUT ###
import pandas as pd
input_data = pd.read_csv('SAMPLE_DATA.csv')

{

        Write codes...

    Training model name : model
    ex) y_pred = model.predict(X_test)


}


### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)
"""

'\n입출력 양식을 \n준수해 주십시오\n\n###  INPUT ###\nimport pandas as pd\ninput_data = pd.read_csv(\'SAMPLE_DATA.csv\')\n\n{\n    \n        Write codes...\n    \n    Training model name : model\n    ex) y_pred = model.predict(X_test)\n    \n\n}\n\n\n### OUTPUT ###\nprint("RMSE:", rmse)\nprint("R2_score:", r2score)\n'

In [18]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # 이 줄은 아래 HistGradientBoostingRegressor를 사용하기 전에 필요합니다.
from sklearn.ensemble import HistGradientBoostingRegressor

In [19]:
###  INPUT ###
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')
input_data.shape

(84840, 50)

In [20]:
df = input_data.drop(columns=['frmDist'])
df = df.sort_values(by='date')

# 데이터를 훈련 세트와 테스트 세트로 분할
'''
데이터를 훈련 세트와 테스트 세트로 나누는 데 사용됩니다.
이는 모델의 성능을 평가하기 위해 데이터를 분리하는 일반적인 절차입니다.
'''
# 데이터셋을 data, target으로 변수분리
X = df[df.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum']).columns]
Y = df[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [21]:
class SimpleEnsemble(BaseEstimator, RegressorMixin):
    def __init__(self, models):
        self.models = [MultiOutputRegressor(model) for model in models]

    def fit(self, X, y):
        for model in self.models:
            model.fit(X, y)
        return self

    def predict(self, X):
        predictions = np.array([model.predict(X) for model in self.models])
        return np.mean(predictions, axis=0)

#linearRegressor = LinearRegression()
#lassocv = LassoCV()
#ridge = Ridge()
#lassolarscv = LassoLarsCV()
#elasticnetcv = ElasticNetCV()
model_xgb = xgb.XGBRegressor()
GBoost = GradientBoostingRegressor()
LightGB = lgb.LGBMRegressor()
RF = RandomForestRegressor()
ETR = ExtraTreesRegressor()
DT = DecisionTreeRegressor()
HGBR = HistGradientBoostingRegressor()
#AdaBoost = AdaBoostRegressor()

# 테스트 세트에 대한 예측
models = [GBoost, LightGB, RF, model_xgb, ETR, DT, HGBR]
model = SimpleEnsemble(models)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025502 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9000
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 39
[LightGBM] [Info] Start training from score 25537.923871
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.025306 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9000
[LightGBM] [Info] Number of data points in the train set: 67872, number of used features: 39
[LightGBM] [Info] Start training from score 144306.559980


In [22]:
'''
mean_squared_error와 r2_score는 회귀 모델의 예측 성능을 평가하는 메트릭입니다.
RMSE는 예측값과 실제값 간의 제곱 오차의 평균 제곱근을 나타내며,
R2 스코어는 모델이 데이터의 분산을 얼마나 설명하는지를 나타냅니다.
'''
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2score = r2_score(y_test, y_pred)


### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)

RMSE: 26800.775657675393
R2_score: 0.997519458957072
